In [1]:
import os
import gc
import time
import codecs
from datetime import date, datetime

random_seed = 42

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import xgboost as xgb

%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
mpl.rc('lines', linewidth=4, color='g')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import keras
import tensorflow as tf
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import (Dense, Activation, Dropout, Flatten, Bidirectional,
                          CuDNNGRU, Embedding, Conv1D, CuDNNLSTM,                           
                          MaxPooling1D, GlobalMaxPooling1D, LSTM) 
from keras.utils import plot_model, np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [14]:
from git import Repo

PATH_OF_GIT_REPO = '/home/hq/Kaggle/.git'

def git_push(COMMIT_MESSAGE):
    try:
        repo = Repo(PATH_OF_GIT_REPO)
        repo.git.add(update=True)
        repo.index.commit(COMMIT_MESSAGE)
        origin = repo.remote(name='Kagglez')
        origin.push()
    except:
        print('Some error occured while pushing the code')

In [13]:
for dirname, _, filenames in os.walk('../Input/nfl-2020'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../Input/nfl-2020/train.csv
../Input/nfl-2020/kaggle/competitions/nflrush/sample_submission.csv.encrypted
../Input/nfl-2020/kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so
../Input/nfl-2020/kaggle/competitions/nflrush/__init__.py
../Input/nfl-2020/kaggle/competitions/nflrush/test.csv.encrypted


In [15]:
git_push('easy_commits')

Some error occured while pushing the code
